# Análisis Exploratorio de Datos - Proyecto Big Data

Este cuaderno demuestra la conexión a la base de datos MongoDB y realiza un análisis básico de los documentos recolectados.

## 1. Configuración e Importaciones

In [ ]:
import os
import pandas as pd
from pymongo import MongoClient
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

MONGO_URI = os.getenv('MONGO_URI')
MONGO_DB = os.getenv('MONGO_DB', 'proyecto_big_data')
MONGO_COLLECTION = os.getenv('MONGO_COLLECTION', 'documentos')

print("Librerías importadas y configuración cargada.")

## 2. Conexión a MongoDB

In [ ]:
try:
    client = MongoClient(MONGO_URI)
    db = client[MONGO_DB]
    collection = db[MONGO_COLLECTION]
    
    total_docs = collection.count_documents({})
    print(f"Conexión exitosa. Total de documentos en la colección: {total_docs}")
except Exception as e:
    print(f"Error de conexión: {e}")

## 3. Extracción de Datos para Análisis

In [ ]:
# Obtener documentos (limitado a campos relevantes)
cursor = collection.find({}, {
    "titulo": 1, 
    "tipo": 1, 
    "tamano_mb": 1, 
    "fecha_descarga": 1, 
    "metadatos.categoria": 1
})

# Convertir a DataFrame
df = pd.DataFrame(list(cursor))

# Aplanar metadatos si es necesario
if not df.empty and 'metadatos' in df.columns:
    df['categoria'] = df['metadatos'].apply(lambda x: x.get('categoria') if isinstance(x, dict) else 'Desconocido')

df.head()

## 4. Estadísticas Básicas

In [ ]:
print("Distribución por Tipo de Archivo:")
print(df['tipo'].value_counts())

print("\nDistribución por Categoría:")
print(df['categoria'].value_counts())